 # 2DMD at a Glance

The commands below import the libraries used throughout the Jupyter notebook

In [ ]:
import ast
import gzip
import io
import os
import re
import requests
import tarfile
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter, OrderedDict
from itertools import chain
from joblib import Parallel, delayed
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import Structure
from pymatgen.core.periodic_table import Element
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

## Functions

Functions for data processing and data preparation in the Pandas DataFrame format

In [ ]:
BASES_and_CONCENTRATIONS = [['BP_spin_500', 'high'], ['GaSe_spin_500', 'high'],
                            ['hBN_spin_500', 'high'], ['InSe_spin_500', 'high'],
                            ['MoS2', 'low'], ['MoS2_500', 'high'],
                            ['WSe2', 'low'], ['WSe2_500', 'high']
                           ]


def readStructures(base, concentration, structure_ids):
    
    with zipfile.ZipFile('2d-materials-point-defects-all.zip') as datasets_zip:
        
        base_in_zip = f'{concentration}_density_defects/{base}'
        with tarfile.open(
            fileobj=io.BytesIO(datasets_zip.read(f'{base_in_zip}/initial.tar.gz'))
        ) as tf:
            cifs = [tf.extractfile(f'{_id}.cif').read().decode("utf-8") 
                    for _id in tqdm(structure_ids)]

    return pd.DataFrame(cifs, index=structure_ids, columns=['cif'])

def parseStructure(cif_string):

    structure = Structure.from_str(cif_string, fmt='cif') #.get_structures(primitive=False)[0]
    space_group_number = SpacegroupAnalyzer(structure).get_space_group_number()
    lattice = structure.lattice.matrix
    positions = np.array([x.coords for x in structure])
    atomic_symbols = np.array(list(map(lambda x: x.symbol, structure.species)))
    atomic_numbers = np.array(list(map(lambda x: Element(x).number, atomic_symbols)))
    formula = OrderedDict(sorted(Counter(atomic_symbols).items()))
    return [space_group_number, lattice, positions, atomic_symbols, atomic_numbers, formula, 
                         len(atomic_numbers), len(formula)]


def structureFormationEnergy(structure, neat_elements):

    fe = structure['energy']
    formula = structure['formula']
    for element in formula.keys() :
        element_chem_potential = neat_elements.loc[element]['chemical_potential']
        fe -= formula[element] * element_chem_potential
    return fe


def flatList(list_):
    
    return [str(item) for sublist in list_ for item in sublist]


def readDataFromArchive(base, concentration):

    with zipfile.ZipFile('2d-materials-point-defects-all.zip') as datasets_zip:

        base_in_zip = f'{concentration}_density_defects/{base}'

        with datasets_zip.open(f'{base_in_zip}/defects.csv.gz') as defects_gz :
            with gzip.open(defects_gz) as defects_data :
                defects_df = pd.read_csv(defects_data, sep=',')

        with datasets_zip.open(f'{base_in_zip}/descriptors.csv') as descriptors :
            descriptors_df = pd.read_csv(descriptors, sep=',')

        with datasets_zip.open(f'{base_in_zip}/elements.csv') as elements :
            neat_elements_df = pd.read_csv(elements, sep=',')
        neat_elements_df.set_index('element', inplace=True, drop=True)
        print(neat_elements_df)

        with datasets_zip.open(f'{base_in_zip}/initial_structures.csv') as init_structures :
            init_structures_df = pd.read_csv(init_structures, sep=',')

        with datasets_zip.open(f'{base_in_zip}/targets.csv.gz') as targets_gz :
            with gzip.open(targets_gz) as targets_data :
                targets_df = pd.read_csv(targets_data, sep=',')

    return defects_df, descriptors_df, neat_elements_df, init_structures_df, targets_df


def handleTableData(base, concentration) :

    defects_df, descriptors_df, neat_elements_df, init_structures_df, targets_df = \
                                                        readDataFromArchive(base, concentration)
    print('\n' + '*'*90 + '\n')
    
    print(f'Information on defects.csv.gz for {base}_{concentration}_concentration')
    for unused_property in ['Unnamed: 0', 'defect_id'] :
        if unused_property in defects_df.columns:
            defects_df.drop(unused_property, axis=1, inplace=True)
    print('\tcolumns:', defects_df.columns)
    print('\tshape:', defects_df.shape)
    number_of_unique = len(defects_df['_id'].unique())
    print(f'\t{number_of_unique} unique defect structures')
    number_of_unique = len(defects_df['descriptor_id'].unique())
    print(f'\t{number_of_unique} unique defect descriptors')

    print(f'Information on descriptors.csv for {base}_{concentration}_concentration')
    for unused_property in ['Unnamed: 0', 'defect_id'] :
        if unused_property in descriptors_df.columns:
            descriptors_df.drop(unused_property, axis=1, inplace=True)
    print('\tcolumns:', descriptors_df.columns)
    print('\tshape:', descriptors_df.shape)
    number_of_unique = len(descriptors_df['_id'].unique())
    print(f'\t{number_of_unique} unique defect descriptors')

    print(f'Information on targets.csv for {base}_{concentration}_concentration')
    for unused_property in ['Unnamed: 0', 'defect_id'] :
        if unused_property in targets_df.columns:
            targets_df.drop(unused_property, axis=1, inplace=True)
    print('\tcolumns:', targets_df.columns)
    print('\tshape:', targets_df.shape)
    number_of_unique = len(targets_df['_id'].unique())
    print(f'\t{number_of_unique} unique defect structures')
    number_of_unique = len(targets_df['descriptor_id'].unique())
    print(f'\t{number_of_unique} unique defect descriptors')

    print('\n' + '*'*90 + '\n')
    print('Data checking...')
    print(f'Is \'descriptor_id\'s (defects.csv) a subset of \'_id\'s (descriptors.csv)?', 
          set(defects_df['descriptor_id']).issubset(set(descriptors_df['_id'])))

    defects_df.set_index('_id', inplace=True, drop=True)
    targets_df.set_index('_id', inplace=True, drop=True)
    descriptors_df.set_index('_id', inplace=True, drop=True)        

    data = pd.DataFrame(descriptors_df.loc[defects_df['descriptor_id']].values, 
                                    columns=descriptors_df.columns, index=defects_df.index)

    for additional_data in [defects_df, targets_df] :
        for piece_of_data in additional_data.columns :
            if (piece_of_data in data.columns) and (additional_data[piece_of_data].dtype != 'O') :
                diff = additional_data[piece_of_data] - data[piece_of_data]
                out_ = np.abs(diff).max()
                print(f'{piece_of_data:>21s} is in the data with the maximum difference of {out_}')
            elif (piece_of_data in data.columns) and (additional_data[piece_of_data].dtype == 'O') :
                out_ = all(data[piece_of_data] == additional_data[piece_of_data])
                print(f'{piece_of_data:>21s} is in the data. Are the column contents the same? {out_}')
            else:
                data[piece_of_data] = additional_data[piece_of_data]

    print('\n' + '*'*90 + '\n')
    cells_available = sorted(data['cell'].unique())
    print('Unique supercells:', cells_available)
    print('\n' + '*'*90 + '\n')
    
    data['defects'] = data['defects'].apply(ast.literal_eval)
    data['defect_nsites'] = data['defects'].apply(len)
                    
    return (data, neat_elements_df, init_structures_df)


def prepareBaseConcentrationDataset(base, concentration, save_to_file=False) :

    data, neat_elements, _ = handleTableData(base, concentration)

    print('Reading structures...')
    structures = readStructures(base, concentration, data.index)
    data = pd.concat((data, structures), axis=1)

    # parse structural info
    print('Parsing structural information...')
    additional_data = Parallel(n_jobs=-1)(delayed(parseStructure)\
                                     (data.loc[_id].cif) 
                                     for _id in tqdm(data.index))
    additional_data = pd.DataFrame(additional_data, index=data.index, 
                        columns=['space_group_no', 'lattice', 'atomic_positions', 'atomic_symbols', 
                                           'atomic_numbers', 'formula', 'nsites', 'nspecies'])

    data = pd.concat((data, additional_data), axis=1)

    # collect additional data
    data['defect_concentration'] = concentration
    data['structure_formation_energy'] = [structureFormationEnergy(data.loc[i], neat_elements) 
                                             for i in data.index]
    data['structure_formation_energy_per_atom'] = data['structure_formation_energy'] / data['nsites']
    data['composition_string'] = data['formula'].apply(lambda x: '_'.join(flatList(list(x.items()))))

    data = data[sorted(data.columns)]
    
    if save_to_file :
        data.to_pickle(f'2d-base_{base}-{concentration}_defect_concentration-table.pkl.gz')
    
    return data


def prepareFullDataset(save_to_file=True, filename='2d-materials-point-defects-all-table.pkl.gz'):

    base_datasets = [prepareBaseConcentrationDataset(base, concentration) 
                         for base, concentration in BASES_and_CONCENTRATIONS]
    
    full_dataset = pd.concat(base_datasets, axis=0)

    full_dataset['ordinal_id'] = list(map(float, range(full_dataset.shape[0])))
    full_dataset = full_dataset[sorted(full_dataset.columns)]
    print(f'Full dataset shape is {full_dataset.shape}')
    
    if save_to_file:
        full_dataset.to_pickle(filename)
        return None
    
    else:
        return full_dataset

## Download and Processing Datasets

This part serves to download the dataset and transform it into a pandas dataframe.

The code of the cell below can be executed <ins>once</ins> and will result in two files being created in the folder it was run from, namely *'2d-materials-point-defects-all.zip'* (the original data) and *'2d-materials-point-defects-all-table.pkl.gz'* (data in table format for further use). 
On a MacBook Pro (Intel Core i7 12 threads), data collection takes about 3 minutes for the first run.
On subsequent runs, the cell reports the presence of the data in table format and does not repeat processing (if necessary, delete the file and repeat processing).

**Reminder**: If you use the 2DMD dataset, please cite the following paper:

Huang, P., Lukin, R., Faleev, M. et al. Unveiling the complex structure-property correlation of defects in 2D materials based on high throughput datasets. npj 2D Mater Appl 7, 6 (2023). https://doi.org/10.1038/s41699-023-00369-1

In [ ]:
%%time
if not os.path.isfile('2d-materials-point-defects-all-table.pkl.gz') :
    url_main = 'https://constructor.app/platform/open/2d-materials-point-defects/attachments
    url_attach = 'b62e17e599364a12941a9ae494a14736?name=2d-materials-point-defects-all.zip'
    request = requests.get('/'.join([url_main, url_attach]), allow_redirects=True)
    open('2d-materials-point-defects-all.zip', 'wb').write(request.content)
    print('\nThe dataset has been successfully downloaded and is currently being processed!\n')

    prepareFullDataset()
    
else :
    print('\nThe dataset was downloaded and processed previously!\n')

## Handle data

### fast screening routines
quick procedures are aimed at obtaining general information about the data

In [ ]:
full_dataset = pd.read_pickle('2d-materials-point-defects-all-table.pkl.gz')
print(f'Dataset shape (objects, features) is {full_dataset.shape}')
print(f'Number of unique structure ids is {len(full_dataset.index.unique())}')

In [ ]:
full_dataset.info()

In [ ]:
full_dataset.describe()

In [ ]:
full_dataset.groupby(['base', 'cell', 'defect_concentration', 'defect_nsites',]).size()

In [ ]:
sns.pairplot(full_dataset[['structure_formation_energy',
                           'structure_formation_energy_per_atom',
                           'nsites', 'defect_nsites', 
                           'homo_lumo_gap_max', 'homo_lumo_gap_min',
                           'base']], 
                         hue='base', palette='viridis')

In [ ]:
sub_data = full_dataset[(full_dataset['base'] == 'WSe2') | (full_dataset['base'] == 'MoS2')]
sns.pairplot(sub_data[['formation_energy', 'structure_formation_energy',
                       'nsites', 'defect_nsites', 'base']], 
             hue='base', palette='viridis')

In [ ]:
sub_data = full_dataset[(full_dataset['base'] != 'WSe2') & (full_dataset['base'] != 'MoS2')]
sns.pairplot(sub_data[['formation_energy', 'structure_formation_energy',
                       'nsites', 'defect_nsites', 'base']], 
             hue='base', palette='viridis')

### defects and space groups
the obtained defect/symmetry relations for the WSe<sub>2</sub> and MoS<sub>2</sub> base materials

In [ ]:
full_dataset[full_dataset['base']=='WSe2'].groupby(['defect_nsites','space_group_no']).size()

In [ ]:
full_dataset[full_dataset['base']=='MoS2'].groupby(['defect_nsites','space_group_no']).size()

### target analysis
visual and numerical analysis of target (thermodynamic) properties

In [ ]:
for base in ['WSe2', 'MoS2'] :
    sub_data = full_dataset[(full_dataset['base'] == base) & (full_dataset['defect_nsites'] >= 3)]
    sns.pairplot(sub_data[['structure_formation_energy', 'formation_energy', 
                           'space_group_no','defect_nsites']],
                 hue='defect_nsites', palette='flare', corner=True)
    # plt.title(base)
    plt.show()
    sub_data = full_dataset[(full_dataset['base'] == base) & (full_dataset['defect_nsites'] >= 3)]
    sns.pairplot(sub_data[['structure_formation_energy_per_atom', 'formation_energy_per_site',
                           'space_group_no','defect_nsites']],
                 hue='defect_nsites', palette='flare', corner=True)
    plt.title(base)
    plt.show()
    

In [ ]:
for base in ['WSe2', 'MoS2'] :
    result = []
    targets_to_analize = ['structure_formation_energy', 'structure_formation_energy_per_atom',
                          'formation_energy', 'formation_energy_per_site', ]
    defect_contents = sorted(full_dataset[full_dataset['base'] == base]['defect_nsites'].unique())
    for defect_ns in defect_contents :
        subset = full_dataset[(full_dataset['base'] == base) & \
                                      (full_dataset['defect_nsites'] == defect_ns)][targets_to_analize]
        result += [[subset[target].std() for target in targets_to_analize]]

    result = np.array(result)
    result = result.T

    for i in range(result.shape[0]) :
        if i == 0 :
            plt.plot(result[i],  'o-', lw=2, label=targets_to_analize[i],)
        else :
            plt.plot(result[i], lw=2, label=targets_to_analize[i])
    plt.grid(alpha=0.3)
    plt.legend()
    plt.yscale('log')
    plt.xlabel('defect positions')
    plt.ylabel('std of target property, a.u.')
    plt.title(base)
    plt.xticks(range(len(defect_contents)), list(map(str, defect_contents)))
    plt.show()

### Sample input for Graph Neural Networks (Allegro)
the code below represents a way to prepare the MoS2 (or alternatively WSe2) subsets for training, validating, and testing a graph neural network and includes visual control of target property distributions

In [ ]:
def allegroInput(dataset_file) :
    
    print(f'Allegro input generation for {dataset_file}')
    data = pd.read_pickle(dataset_file)
    dataset_xyz = re.sub('pkl.gz', 'xyz', dataset_file)
    with open(dataset_xyz, 'w') as out :
        for index in tqdm(data.index) :
            item = data.loc[index]
            lattice = item['lattice']
            atomic_symbols = item['atomic_symbols']
            positions = item['atomic_positions']
            fe = item['structure_formation_energy']
            fe_pa = item['structure_formation_energy_per_atom']
            fe_ori = item['formation_energy']
            ordinal_id = item['ordinal_id']

            out.write(f'{item.nsites}\n')
            out.write('Lattice=\"'+' '.join(list(map(str, list((lattice).reshape(-1,)))))+'\" ')
            out.write('Properties=species:S:1:pos:R:3 pbc=\"T T F\"  ') 
            out.write(f'ordinal_id={ordinal_id} \
                        structure_formation_energy={fe} \
                        structure_formation_energy_per_atom={fe_pa} \
                        original_formation_energy={fe_ori}')        
            out.write('\n')
            for symbol, pos in zip(atomic_symbols, positions) :
                out.write(symbol + '   ' + ' '.join(list(map(str, pos)))+'\n')

In [ ]:
if not os.path.isdir('for_allegro/') :
    os.mkdir('for_allegro/')
    
base = 'MoS2' # base to be processed
base_dataset = full_dataset[full_dataset['base'] == base].copy()

# selection of high symmetry structures among low defect contents
high_sym_base = base_dataset[(base_dataset['defect_nsites'] < 4) & \
                                     (base_dataset['space_group_no'] >= 8)].copy()

print('train_val_high\n', high_sym_base.shape, '\n',
      high_sym_base.groupby(['defect_nsites', 'space_group_no']).size())

# train/validation (high symmetry structures) split among low defect contents
train_high, val_high = train_test_split(high_sym_base, test_size=0.2, shuffle=True, 
                              stratify=high_sym_base['defect_nsites'], random_state=0)

train_high.to_pickle(f'for_allegro/{base}_high_sym_train.pkl.gz')
val_high.to_pickle(f'for_allegro/{base}_high_sym_val.pkl.gz')
print()

# test subset (250 low symmetry structures) among low defect contents
low_sym_base = base_dataset[(base_dataset['defect_nsites'] < 4) & \
                            (base_dataset['space_group_no'] < 8)].copy()
_, test_low = train_test_split(low_sym_base, test_size=250, shuffle=True, 
                              stratify=low_sym_base['defect_nsites'], random_state=0)
test_low.to_pickle(f'for_allegro/{base}_low_def_test.pkl.gz')

print('test_low\n', test_low.shape, '\n',
      test_low.groupby(['defect_nsites', 'space_group_no']).size())

# test subset (250 low symmetry structures) among high defect contents
high_def_cont = base_dataset[base_dataset['defect_nsites'] >= 4].copy()
_, test_high = train_test_split(high_def_cont, test_size=250, shuffle=True, 
                          stratify=high_def_cont['defect_nsites'], random_state=0)
test_high.to_pickle(f'for_allegro/{base}_high_def_test.pkl.gz')

print('test_high\n', test_high.shape, '\n',
      test_high.groupby(['defect_nsites', 'space_group_no']).size())

print('test.shape', pd.concat((test_low, test_high)).shape)
print()

# train/validation (low symmetry structures) split among low defect contents
# the structures comprising the test subsets above are subtracted from traininig/validation ones
not_test_data = low_sym_base[~np.isin(low_sym_base.index, test_low.index)]
not_test_data = not_test_data.sample(n=high_sym_base.shape[0], random_state=0)

print('train_val_low\n', not_test_data.shape, '\n',
      not_test_data.groupby(['defect_nsites', 'space_group_no']).size())
train_low, val_low = train_test_split(not_test_data, test_size=0.2, shuffle=True, 
                              stratify=not_test_data['defect_nsites'], random_state=0)

train_low.to_pickle(f'for_allegro/{base}_low_sym_train.pkl.gz')
val_low.to_pickle(f'for_allegro/{base}_low_sym_val.pkl.gz')
print()


for energy_type in ['structure_formation_energy_per_atom', 'structure_formation_energy',
                    'formation_energy'] :
    per_atom_addon = '/atom' if energy_type.endswith('_per_atom') else ''
    fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10, 3))
    plt.suptitle(f'TEST: Energy type: {energy_type}')
    
    axs[0].hist([data[energy_type] for data in [train_high, val_high, test_low, test_high]], 
                label=['train', 'val', 'test_low', 'test_high'])
    axs[0].set_xlabel(f'{energy_type}, eV{per_atom_addon}')
    axs[0].set_ylabel('Number of structures')
    axs[0].legend()
    axs[0].grid(alpha=0.3)
    axs[0].set_title('High symmetry train/val')

    axs[1].hist([data[energy_type] for data in [train_low, val_low, test_low, test_high]])
    axs[1].set_xlabel(f'{energy_type}, eV{per_atom_addon}')
    axs[1].grid(alpha=0.3)
    axs[1].set_title('Low symmetry train/val')
    plt.tight_layout()
    plt.show()
    
# convert subsets of data to xyz format
allegroInput(dataset_file=f'for_allegro/{base}_high_sym_train.pkl.gz')
allegroInput(dataset_file=f'for_allegro/{base}_high_sym_val.pkl.gz')
allegroInput(dataset_file=f'for_allegro/{base}_low_sym_train.pkl.gz')
allegroInput(dataset_file=f'for_allegro/{base}_low_sym_val.pkl.gz')
allegroInput(dataset_file=f'for_allegro/{base}_low_def_test.pkl.gz')
allegroInput(dataset_file=f'for_allegro/{base}_high_def_test.pkl.gz')